In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
from scipy.signal import savgol_filter
import os


# Define the inclination_correction and radial_distance functions
def inclination_correction(major, minor, emax, method):
    if minor > major:
        major, minor = minor, major
        
    eccentricity = np.sqrt(1 - (minor / major) ** 2)
    if method == "sine":
        vel_multiplier = np.sin(np.pi / 2 * (eccentricity / emax))
    else:
        vel_multiplier = np.cos(np.pi / 2 * (1 - eccentricity / emax))
    return vel_multiplier


def radial_distance(galax_x, galax_y, star_x, star_y):
    delta_x = star_x - galax_x
    delta_y = galax_y - star_y
    distance = np.sqrt((delta_x) ** 2 + (delta_y) ** 2)
    return distance


# Load data from the CSV files
data = pd.read_csv('/Users/kobibrown/Desktop/Distance_Ladder_Project/clustered_star_data.csv')
results_df = pd.read_csv('/Users/kobibrown/Desktop/Distance_Ladder_Project/Cluster_Ellipse_Data.csv')


def plot_rotation_curve(cluster_number, output_folder):
    # Filter the galaxy_data DataFrame for the desired cluster
    filtered_data = data.loc[data['cluster'] == cluster_number].copy()

    # account for the proper motion of the galaxy
    filtered_data.loc[:, 'RadialVelocity'] -= filtered_data['RadialVelocity'].median()

    # pull ellipse data
    ellipse_data = results_df.loc[results_df['cluster'] == cluster_number]
    major_axis = ellipse_data['major_axis'].values[0]
    minor_axis = ellipse_data['minor_axis'].values[0]

    # correct the star velocities
    emax = np.sqrt(1 - (0.1143347 / 0.4066884) ** 2)  # Maximum eccentricity chosen as edge-on cluster 5
    velocity_correction = 1 / inclination_correction(major_axis, minor_axis, emax, "cos")
    corrected_velocities = abs(filtered_data['RadialVelocity'] * velocity_correction)

    # find the position of each star relative to the centre
    center_x = ellipse_data['center_x'].values[0]
    center_y = ellipse_data['center_y'].values[0]
    radii = [radial_distance(center_x, center_y, x, y) for x, y in zip(filtered_data['Equat'], filtered_data['Polar'])]

    # normalize the radial distances
    normalized_radii = np.array(radii) / max(radii)

    # average the corrected radial velocities over the x values
    n_bins = 5
    bin_means, bin_edges = np.histogram(normalized_radii, bins=n_bins, weights=corrected_velocities)
    bin_count, _ = np.histogram(normalized_radii, bins=n_bins)
    bin_count = bin_count.astype(float)
    bin_count[bin_count == 0] = np.nan
    bin_means = np.divide(bin_means, bin_count)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    #  line of best fit
    best_fit_spline = UnivariateSpline(bin_centers, bin_means, s=0)
    x_vals = np.concatenate(([0], np.linspace(0, max(normalized_radii), 100)))

    n_bins = 25
    ceiling_radii, ceiling_velocities = ceiling_fit_radii_velocities(normalized_radii, corrected_velocities, n_bins)

    # smoothing algo
    window_size, polyorder = 20, 5
    smoothed_ceiling_radii, smoothed_ceiling_velocities = smooth_curve(ceiling_radii, ceiling_velocities, window_size, polyorder)

    # force the origin start
    smoothed_ceiling_radii = np.insert(smoothed_ceiling_radii, 0, 0)
    smoothed_ceiling_velocities = np.insert(smoothed_ceiling_velocities, 0, 0)

    # Create the rotation curve plot
    plt.figure(figsize=(10, 6), dpi=100)
    plt.scatter(normalized_radii, corrected_velocities, c=filtered_data['RadialVelocity'], cmap='RdBu', s=10, label='Stars')
    plt.plot(smoothed_ceiling_radii, smoothed_ceiling_velocities, 'k-', linewidth=2, label='Rotation Curve')
    plt.xlabel('Normalized Radial Distance')
    plt.ylabel('Corrected Radial Velocity (km/s)')
    plt.colorbar(label='Observed Radial Velocity (km/s)')
    plt.title(f'Rotation Curve for Cluster {cluster_number}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.autoscale(enable=True, axis='both', tight=True)
    plt.savefig(os.path.join(output_folder, f'rotation_curve_cluster_{cluster_number}.png'))
    plt.close()

def smooth_curve(x, y, window_size, polyorder):
    if len(y) < window_size:
        window_size = len(y) - 1 if len(y) % 2 == 0 else len(y)
    smoothed_y = savgol_filter(y, window_size, polyorder)
    return x, smoothed_y

# Set the output folder
output_folder = "/Users/kobibrown/Desktop/Distance_Ladder_Project/Rotation_Curves"

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get all unique cluster numbers
unique_clusters = data['cluster'].unique()

# Loop over all clusters and export pngs of all the rotation curves to the specified folder
for cluster_number in unique_clusters:
    plot_rotation_curve(cluster_number, output_folder)



In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import PchipInterpolator
from scipy import stats
import os



# Define the inclination_correction and radial_distance functions
def inclination_correction(major, minor, emax, method):
    if minor > major:
        major, minor = minor, major
        
    eccentricity = np.sqrt(1 - (minor / major) ** 2)
    if method == "sine":
        vel_multiplier = np.sin(np.pi / 2 * (eccentricity / emax))
    else:
        vel_multiplier = np.cos(np.pi / 2 * (1 - eccentricity / emax))
    return vel_multiplier


def radial_distance(galax_x, galax_y, star_x, star_y):
    delta_x = star_x - galax_x
    delta_y = galax_y - star_y
    distance = np.sqrt((delta_x) ** 2 + (delta_y) ** 2)
    return distance

def ceiling_fit_radii_velocities(x, y, n_bins, zscore_threshold=2, iqr_multiplier=1.5):
    # Remove outliers using the IQR method for both x and y
    cleaned_x, cleaned_y = remove_velocity_outliers_iqr(x, y, iqr_multiplier)
    cleaned_x, cleaned_y = remove_velocity_outliers_iqr(cleaned_x, cleaned_y, iqr_multiplier)

    # Bin the data using the max statistic
    bin_means, bin_edges, binnumber = stats.binned_statistic(cleaned_x, cleaned_y, statistic='max', bins=n_bins)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    # Remove bins with NaN (occurs when there's no data in the bin)
    non_nan_indices = np.where(~np.isnan(bin_means))
    cleaned_bin_centers = bin_centers[non_nan_indices]
    cleaned_bin_means = bin_means[non_nan_indices]

    return cleaned_bin_centers, cleaned_bin_means



# Load data from the CSV files
data = pd.read_csv('/Users/kobibrown/Desktop/Distance_Ladder_Project/clustered_star_data.csv')
results_df = pd.read_csv('/Users/kobibrown/Desktop/Distance_Ladder_Project/Cluster_Ellipse_Data.csv')


def plot_rotation_curve(cluster_number, output_folder):
    # Filter the galaxy_data DataFrame for the desired cluster
    filtered_data = data.loc[data['cluster'] == cluster_number].copy()

    # account for the proper motion of the galaxy
    filtered_data.loc[:, 'RadialVelocity'] -= filtered_data['RadialVelocity'].median()

    # pull ellipse data
    ellipse_data = results_df.loc[results_df['cluster'] == cluster_number]
    major_axis = ellipse_data['major_axis'].values[0]
    minor_axis = ellipse_data['minor_axis'].values[0]

    # correct the star velocities
    emax = np.sqrt(1 - (0.1143347 / 0.4066884) ** 2)  # Maximum eccentricity chosen as edge-on cluster 5
    velocity_correction = 1 / inclination_correction(major_axis, minor_axis, emax, "cos")
    corrected_velocities = abs(filtered_data['RadialVelocity'] * velocity_correction)

    # Convert corrected_velocities to a numpy array
    corrected_velocities = corrected_velocities.to_numpy()

    # Convert corrected_velocities back to a pandas Series
    corrected_velocities = pd.Series(corrected_velocities)



    # find the position of each star relative to the centre
    center_x = ellipse_data['center_x'].values[0]
    center_y = ellipse_data['center_y'].values[0]
    radii = [radial_distance(center_x, center_y, x, y) for x, y in zip(filtered_data['Equat'], filtered_data['Polar'])]

    # normalize the radial distances
    normalized_radii = np.array(radii) / max(radii)

    # average the corrected radial velocities over the x values
    n_bins = 5
    ceiling_radii, ceiling_velocities = ceiling_fit_radii_velocities(normalized_radii, corrected_velocities, n_bins)

    if len(ceiling_radii) < 2 or len(ceiling_velocities) < 2:
        print(f"Skipping cluster {cluster_number} due to insufficient data for interpolation.")
        return

    # Add the origin to the ceiling_radii and ceiling_velocities arrays
    ceiling_radii = np.concatenate(([0], ceiling_radii))
    ceiling_velocities = np.concatenate(([0], ceiling_velocities))

    # Create the PCHIP interpolator
    pchip_interpolator = PchipInterpolator(ceiling_radii, ceiling_velocities)
    x_vals = np.linspace(0, max(normalized_radii), 100)
    pchip_y = pchip_interpolator(x_vals)

    # Clip the pchip_y values to ensure the curve doesn't go beyond the plot's limits
    max_ceiling_velocity = np.max(ceiling_velocities)
    pchip_y = np.clip(pchip_y, 0, max_ceiling_velocity)


    # Create the rotation curve plot
    plt.figure(figsize=(10, 6), dpi=100)
    plt.scatter(normalized_radii, corrected_velocities, c=filtered_data['RadialVelocity'], cmap='RdBu', s=10, label='Stars')
    plt.plot(x_vals, pchip_y, 'k-', linewidth=2, label='Rotation Curve')

    plt.xlabel('Normalized Distance from the Centre of The Galaxy', fontsize=14, fontweight='bold')
    plt.ylabel('Corrected Radial Velocity (km/s)', fontsize=14, fontweight='bold')
    plt.title(f'Rotation Curve for Cluster {cluster_number}', fontsize=16, fontweight='bold', pad=20)

    cbar = plt.colorbar(label='Observed Radial Velocity (km/s)')
    cbar.ax.set_ylabel('Observed Radial Velocity (km/s)', fontsize=14, fontweight='bold')

    plt.legend(fontsize=12)

    plt.grid(True)
    plt.tight_layout()
    plt.autoscale(enable=True, axis='both', tight=True)

    # Set the y-axis limits to focus on the majority of the data
    q1 = np.percentile(cleaned_corrected_velocities, 25)
    q3 = np.percentile(cleaned_corrected_velocities, 75)
    iqr = q3 - q1
    lower_bound = q1 - 1 * iqr

    # Set the upper bound to be just above the highest point of the fitted line
    highest_fitted_point = np.max(pchip_y)
    padding_factor = 1.1  # Adjust this value to change the headroom above the highest point
    upper_bound = highest_fitted_point * padding_factor

    plt.ylim(lower_bound, upper_bound)

    plt.savefig(os.path.join(output_folder, f'rotation_curve_cluster_{cluster_number}.png'))
    plt.close()



smoothed_ceiling_radii, smoothed_ceiling_velocities = smooth_curve(ceiling_radii, ceiling_velocities, window_size_fraction=0.2, polyorder=5)

def remove_velocity_outliers_iqr(x, y, iqr_multiplier=1.5):
    q1 = np.percentile(y, 25)
    q3 = np.percentile(y, 75)
    iqr = q3 - q1
    lower_bound = q1 - iqr_multiplier * iqr
    upper_bound = q3 + iqr_multiplier * iqr
    good_indices = np.where((y > lower_bound) & (y < upper_bound))
    cleaned_x = x[good_indices]
    cleaned_y = y.iloc[good_indices]  # Use .iloc indexer
    return cleaned_x, cleaned_y


cleaned_normalized_radii, cleaned_corrected_velocities = remove_velocity_outliers_iqr(normalized_radii, corrected_velocities, iqr_multiplier=1.5)



# Set the output folder
output_folder = "/Users/kobibrown/Desktop/Distance_Ladder_Project/Rotation_Curves"

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get all unique cluster numbers
unique_clusters = data['cluster'].unique()

# Loop over all clusters and export pngs of all the rotation curves to the specified folder
for cluster_number in unique_clusters:
    plot_rotation_curve(cluster_number, output_folder)


In [145]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import PchipInterpolator
from scipy import stats
import os
from scipy.interpolate import CubicSpline


# Define the inclination_correction and radial_distance functions
def inclination_correction(major, minor, emax, method):
    if minor > major:
        major, minor = minor, major
        
    eccentricity = np.sqrt(1 - (minor / major) ** 2)
    if method == "sine":
        vel_multiplier = np.sin(np.pi / 2 * (eccentricity / emax))
    else:
        vel_multiplier = np.cos(np.pi / 2 * (1 - eccentricity / emax))
    return vel_multiplier


def radial_distance(galax_x, galax_y, star_x, star_y):
    delta_x = star_x - galax_x
    delta_y = galax_y - star_y
    distance = np.sqrt((delta_x) ** 2 + (delta_y) ** 2)
    return distance

def ceiling_fit_radii_velocities(x, y, n_bins, zscore_threshold=2, iqr_multiplier=1.5):
    # Remove outliers using the IQR method for both x and y
    cleaned_x, cleaned_y = remove_velocity_outliers_iqr(x, y, iqr_multiplier)
    cleaned_x, cleaned_y = remove_velocity_outliers_iqr(cleaned_x, cleaned_y, iqr_multiplier)

    # Bin the data using the max statistic
    bin_means, bin_edges, binnumber = stats.binned_statistic(cleaned_x, cleaned_y, statistic='max', bins=n_bins)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    # Remove bins with NaN (occurs when there's no data in the bin)
    non_nan_indices = np.where(~np.isnan(bin_means))
    cleaned_bin_centers = bin_centers[non_nan_indices]
    cleaned_bin_means = bin_means[non_nan_indices]

    return cleaned_bin_centers, cleaned_bin_means



# Load data from the CSV files
data = pd.read_csv('/Users/kobibrown/Desktop/Distance_Ladder_Project/clustered_star_data.csv')
results_df = pd.read_csv('/Users/kobibrown/Desktop/Distance_Ladder_Project/Cluster_Ellipse_Data.csv')

def fit_log_line_to_ceiling(x, y):
    x = x[1:]  # Exclude the first element (0) to avoid log(0) warning
    y = y[1:]  # Exclude the first element (0) to avoid log(0) warning
    
    # Add an intermediate point between the origin and the first ceiling point
    x = np.concatenate(([0.5 * x[0]], x))
    y = np.concatenate(([0.5 * y[0]], y))
    
    log_x = np.log10(x)
    log_y = np.log10(y)
    cubic_spline_interpolator = CubicSpline(log_x, log_y, bc_type="natural")
    num_points = 100  # Number of points for the log fit
    log_fit_x_vals = np.linspace(min(log_x), max(log_x), num_points)
    log_fit_y = cubic_spline_interpolator(log_fit_x_vals)
    fit_x = np.concatenate(([0], 10**log_fit_x_vals))
    fit_y = np.concatenate(([0], 10**log_fit_y))
    return fit_x, fit_y

def plot_rotation_curve(cluster_number, output_folder):
    # Filter the galaxy_data DataFrame for the desired cluster
    filtered_data = data.loc[data['cluster'] == cluster_number].copy()

    # account for the proper motion of the galaxy
    filtered_data.loc[:, 'RadialVelocity'] -= filtered_data['RadialVelocity'].median()

    # pull ellipse data
    ellipse_data = results_df.loc[results_df['cluster'] == cluster_number]
    major_axis = ellipse_data['major_axis'].values[0]
    minor_axis = ellipse_data['minor_axis'].values[0]

    # correct the star velocities
    emax = np.sqrt(1 - (0.1143347 / 0.4066884) ** 2)  # Maximum eccentricity chosen as edge-on cluster 5
    velocity_correction = 1 / inclination_correction(major_axis, minor_axis, emax, "cos")
    corrected_velocities = abs(filtered_data['RadialVelocity'] * velocity_correction)

    # Convert corrected_velocities to a numpy array
    corrected_velocities = corrected_velocities.to_numpy()

    # Convert corrected_velocities back to a pandas Series
    corrected_velocities = pd.Series(corrected_velocities)



    # find the position of each star relative to the centre
    center_x = ellipse_data['center_x'].values[0]
    center_y = ellipse_data['center_y'].values[0]
    radii = [radial_distance(center_x, center_y, x, y) for x, y in zip(filtered_data['Equat'], filtered_data['Polar'])]

    # normalize the radial distances
    normalized_radii = np.array(radii) / max(radii)

    # average the corrected radial velocities over the x values
    n_bins = 5
    ceiling_radii, ceiling_velocities = ceiling_fit_radii_velocities(normalized_radii, corrected_velocities, n_bins)

    if len(ceiling_radii) < 2 or len(ceiling_velocities) < 2:
        print(f"Skipping cluster {cluster_number} due to insufficient data for interpolation.")
        return

    # Add the origin to the ceiling_radii and ceiling_velocities arrays
    ceiling_radii = np.concatenate(([0], ceiling_radii))
    ceiling_velocities = np.concatenate(([0], ceiling_velocities))

    # Create the PCHIP interpolator
    pchip_interpolator = PchipInterpolator(ceiling_radii, ceiling_velocities)
    x_vals = np.linspace(0, max(normalized_radii), 100)
    pchip_y = pchip_interpolator(x_vals)

    # Clip the pchip_y values to ensure the curve doesn't go beyond the plot's limits
    max_ceiling_velocity = np.max(ceiling_velocities)
    pchip_y = np.clip(pchip_y, 0, max_ceiling_velocity)


    # Fit a smooth log line to the ceiling of the data
    log_fit_x, log_fit_y = fit_log_line_to_ceiling(ceiling_radii, ceiling_velocities)

    # Create the rotation curve plot
    plt.figure(figsize=(10, 6), dpi=100)
    plt.scatter(normalized_radii, corrected_velocities, c=filtered_data['RadialVelocity'], cmap='RdBu', s=10, label='Stars')
    plt.plot(log_fit_x, log_fit_y, 'k-', linewidth=1, label='Rotation Curve')


    plt.xlabel('Normalized Distance from the Centre of The Galaxy', fontsize=14, fontweight='bold')
    plt.ylabel('Corrected Rotational Velocity (km/s)', fontsize=14, fontweight='bold')
    plt.title(f'Rotation Curve for Cluster {cluster_number}', fontsize=16, fontweight='bold', pad=20)

    cbar = plt.colorbar(label='Corrected Radial Velocity (km/s)')
    cbar.ax.set_ylabel('Corrected Radial Velocity (km/s)', fontsize=14, fontweight='bold')

    plt.legend(fontsize=12)

    plt.grid(True)
    plt.tight_layout()
    plt.autoscale(enable=True, axis='both', tight=True)

    # Set the y-axis limits to focus on the majority of the data
    q1 = np.percentile(cleaned_corrected_velocities, 25)
    q3 = np.percentile(cleaned_corrected_velocities, 75)
    iqr = q3 - q1
    lower_bound = q1 - 1 * iqr

    # Set the upper bound to be just above the highest point of the fitted line
    highest_fitted_point = np.max(pchip_y)
    padding_factor = 1.1  # Adjust this value to change the headroom above the highest point
    upper_bound = highest_fitted_point * padding_factor

    plt.ylim(lower_bound, upper_bound)

    plt.savefig(os.path.join(output_folder, f'rotation_curve_cluster_{cluster_number}.png'))
    plt.close()



smoothed_ceiling_radii, smoothed_ceiling_velocities = smooth_curve(ceiling_radii, ceiling_velocities, window_size_fraction=0.2, polyorder=5)

def remove_velocity_outliers_iqr(x, y, iqr_multiplier=1.5):
    q1 = np.percentile(y, 25)
    q3 = np.percentile(y, 75)
    iqr = q3 - q1
    lower_bound = q1 - iqr_multiplier * iqr
    upper_bound = q3 + iqr_multiplier * iqr
    good_indices = np.where((y > lower_bound) & (y < upper_bound))
    cleaned_x = x[good_indices]
    cleaned_y = y.iloc[good_indices]  # Use .iloc indexer
    return cleaned_x, cleaned_y


cleaned_normalized_radii, cleaned_corrected_velocities = remove_velocity_outliers_iqr(normalized_radii, corrected_velocities, iqr_multiplier=1.5)



# Set the output folder
output_folder = "/Users/kobibrown/Desktop/Distance_Ladder_Project/Rotation_Curves"

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get all unique cluster numbers
unique_clusters = data['cluster'].unique()

# Loop over all clusters and export pngs of all the rotation curves to the specified folder
for cluster_number in unique_clusters:
    plot_rotation_curve(cluster_number, output_folder)


In [147]:
# Initialize an empty list to store the maximum velocities for each cluster
max_velocities = []

# Loop over all clusters and find the maximum corrected rotational velocity
for cluster_number in unique_clusters:
    # Filter the galaxy_data DataFrame for the desired cluster
    filtered_data = data.loc[data['cluster'] == cluster_number].copy()

    # account for the proper motion of the galaxy
    filtered_data.loc[:, 'RadialVelocity'] -= filtered_data['RadialVelocity'].median()

    # pull ellipse data
    ellipse_data = results_df.loc[results_df['cluster'] == cluster_number]
    major_axis = ellipse_data['major_axis'].values[0]
    minor_axis = ellipse_data['minor_axis'].values[0]

    # correct the star velocities
    emax = np.sqrt(1 - (0.1143347 / 0.4066884) ** 2)  # Maximum eccentricity chosen as edge-on cluster 5
    velocity_correction = 1 / inclination_correction(major_axis, minor_axis, emax, "cos")
    corrected_velocities = abs(filtered_data['RadialVelocity'] * velocity_correction)

    # Get the maximum velocity for this cluster
    max_velocities.append(corrected_velocities.max())

# Create a DataFrame with columns for cluster numbers and maximum velocities
output_df = pd.DataFrame({'cluster': unique_clusters, 'max_corrected_rotational_velocity': max_velocities})

# Export the DataFrame to a CSV file
output_df.to_csv('/Users/kobibrown/Desktop/Distance_Ladder_Project/max_rotational_velocities.csv', index=False)